In [93]:
from importlib import reload

import task1
reload(task1)
from task1 import *

import task2
reload(task2)
from task2 import *

import countries as ct

path1="data/title.akas.tsv.gz"
path2="data/title.basics.tsv.gz"
path3="data/title.ratings.tsv.gz"

# Results:

In [98]:
ha = hegemonyAnalysis(path1, path2, path3, K = 10000, verbose = True)

Loading data... 

c:\Users\priva\OneDrive\Desktop\STUDIA\Narzędzia Informatyczne w Pythonie wspierające Analizę Danych\Projekt zaliczeniowy\load_data.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path_to_file,


Done!
Preprocessing... Done!
Computing cinematic impacts... Done!
Computing hegemony dataframes... Analyzing gdp...
Country 'Afghanistan' not found in gdp World Bank Database.
Analyzing population...
Analyzing gdp pc...
Country 'Afghanistan' not found in gdp pc World Bank Database.
Done!


In [96]:
ha

{'GDP':                           weak impact strong impact
 Albania                            59            89
 Argentina                         -28           -19
 Australia                         -35           -33
 Austria                           -38           -26
 Bangladesh                        -12            30
 Belgium                           -26           -32
 Brazil                            -15            -5
 Canada                            -24           -44
 China                             -27           -55
 Croatia                            48            44
 Cyprus                             99            55
 Denmark                            20             8
 Ecuador                             8            -3
 El Salvador                        49            39
 Ethiopia                           21            23
 Finland                            30            34
 France                            -14           -33
 Georgia                            64 

# Thinking:

In [5]:
df, df3 = load_dataframes(path1, path2, path3)
df_analysis = prepareDataset(df, df3)

c:\Users\priva\OneDrive\Desktop\STUDIA\Narzędzia Informatyczne w Pythonie wspierające Analizę Danych\Projekt zaliczeniowy\load_data.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path_to_file,


In [37]:
K = 10000
df_analysis = df_analysis.loc[df_analysis["numVotes"] > K]

rank_weak = df_analysis.groupby(by="region")["numVotes"].mean().astype(int)
rank_weak.index = [ct.countries[x] for x in rank_weak.index]
rank_weak = list(rank_weak.sort_values(ascending=False).index)
rank_weak

['United States',
 'Brazil',
 'India',
 'Lithuania',
 'United Arab Emirates',
 'South Korea',
 'Portugal',
 'Indonesia',
 'Iran, Islamic Republic of',
 'Japan',
 'Norway',
 'Hungary',
 'Slovakia',
 'France',
 'South Africa',
 'Timor-Leste',
 'Finland',
 'Latvia',
 'Iceland',
 'Romania',
 'Spain',
 'Sierra Leone',
 'Cyprus',
 'Poland',
 'Hong Kong',
 'Somalia',
 'Germany',
 'Sweden',
 'Mexico',
 'Turkey',
 'United Kingdom',
 'Philippines',
 'Virgin Islands, U.S.',
 'Afghanistan',
 'Denmark',
 'China',
 'Canada',
 'Croatia',
 'Italy',
 'Israel',
 'Netherlands',
 'Ecuador',
 'Ethiopia']

In [38]:
df_analysis["score"] = df_analysis["averageRating"] * df_analysis["numVotes"]
rank_strong = df_analysis.groupby(by="region")["score"].sum()/df_analysis.groupby(by="region")["numVotes"].sum()
rank_strong.index = [ct.countries[x] for x in rank_strong.index]
rank_strong = list(rank_strong.sort_values(ascending=False).index)
rank_strong

['Lithuania',
 'India',
 'Brazil',
 'Croatia',
 'United Arab Emirates',
 'Iran, Islamic Republic of',
 'Sierra Leone',
 'Romania',
 'Poland',
 'Turkey',
 'Ecuador',
 'Hungary',
 'Japan',
 'Israel',
 'Virgin Islands, U.S.',
 'Timor-Leste',
 'Finland',
 'Portugal',
 'Norway',
 'Sweden',
 'South Korea',
 'Denmark',
 'Germany',
 'Italy',
 'Mexico',
 'Philippines',
 'Afghanistan',
 'South Africa',
 'Canada',
 'Indonesia',
 'Hong Kong',
 'Netherlands',
 'France',
 'United Kingdom',
 'Somalia',
 'Spain',
 'Cyprus',
 'Slovakia',
 'United States',
 'Iceland',
 'Latvia',
 'China',
 'Ethiopia']

# Collective analysis

In [97]:
country_list = rank_strong.copy()
country_list.extend(rank_weak)
country_list = list(set(country_list))

rank_dict = {}
for indic in ['GDP', 'Population', 'GDP PC']:
    print(f"Analyzing {indic.lower()}...")
    rank_dict[indic] = {}
    rank = WorldBankRank(indic)
    for country in country_list:
        try:
            impact_weak = rank.index(country) - rank_weak.index(country)
            impact_strong = rank.index(country) - rank_strong.index(country)
        except ValueError:
            print(f"Country '{country}' not found in {indic.lower()} World Bank Database.")
            impact_weak = pd.NA
            impact_strong = pd.NA
            
        rank_dict[indic][country] = {'weak impact': impact_weak, 'strong impact': impact_strong}
    rank_dict[indic] = pd.DataFrame.from_dict(rank_dict[indic]).T.dropna().sort_index()

Analyzing gdp...
Country 'Virgin Islands, U.S.' not found in gdp World Bank Database.
Country 'Afghanistan' not found in gdp World Bank Database.
Analyzing population...
Country 'Virgin Islands, U.S.' not found in population World Bank Database.
Analyzing gdp pc...
Country 'Virgin Islands, U.S.' not found in gdp pc World Bank Database.
Country 'Afghanistan' not found in gdp pc World Bank Database.


In [89]:
rank_dict['GDP']

,weak impact,strong impact
Brazil,7,6
Canada,-27,-19
China,-34,-40
Croatia,37,71
Cyprus,79,65
Denmark,0,13
Ecuador,20,51
Ethiopia,14,14
Finland,30,30
France,-7,-26


# Analysis for GDP

In [64]:
rank_dict = {}
gdp_rank = WorldBankRank("GDP")
for country in country_list:
    try:
        impact_weak = gdp_rank.index(country) - rank_weak.index(country)
        impact_strong = gdp_rank.index(country) - rank_strong.index(country)
    except ValueError:
        print(f"Country '{country}' not found in latest GDP World Bank Database.")
        impact_weak = pd.NA
        impact_strong = pd.NA
        
    rank_dict[country] = {'weak impact': impact_weak, 'strong impact': impact_strong}
    
rank_df = pd.DataFrame.from_dict(rank_dict).T.dropna().sort_index()
rank_df

Country 'Virgin Islands, U.S.' not found in World Bank Database.
Country 'Afghanistan' not found in World Bank Database.


# Analysis for population

In [76]:
rank_dict = {}
for country in country_list:
    try:
        impact_weak = pop_rank.index(country) - rank_weak.index(country)
        impact_strong = pop_rank.index(country) - rank_strong.index(country)
    except ValueError:
        print(f"Country '{country}' not found in latest population World Bank Database.")
        impact_weak = pd.NA
        impact_strong = pd.NA
        
    rank_dict[country] = {'weak impact': impact_weak, 'strong impact': impact_strong}
    
rank_df = pd.DataFrame.from_dict(rank_dict).T.dropna().sort_index()
rank_df

Country 'Virgin Islands, U.S.' not found in latest population World Bank Database.


,weak impact,strong impact
Afghanistan,2,9
Brazil,5,4
Canada,0,8
China,-34,-40
Croatia,91,125
Cyprus,135,121
Denmark,78,91
Ecuador,25,56
Ethiopia,-32,-32
Finland,99,99


# Analysis for GDP P.C.

In [77]:
rank_dict = {}
for country in country_list:
    try:
        impact_weak = gdpPC_rank.index(country) - rank_weak.index(country)
        impact_strong = gdpPC_rank.index(country) - rank_strong.index(country)
    except ValueError:
        print(f"Country '{country}' not found in latest GDP P.C. World Bank Database.")
        impact_weak = pd.NA
        impact_strong = pd.NA
        
    rank_dict[country] = {'weak impact': impact_weak, 'strong impact': impact_strong}
    
rank_df = pd.DataFrame.from_dict(rank_dict).T.dropna().sort_index()
rank_df

Country 'Virgin Islands, U.S.' not found in latest GDP P.C. World Bank Database.
Country 'Afghanistan' not found in latest GDP P.C. World Bank Database.


,weak impact,strong impact
Brazil,78,77
Canada,-21,-13
China,35,29
Croatia,16,50
Cyprus,8,-6
Denmark,-27,-14
Ecuador,59,90
Ethiopia,121,121
Finland,-3,-3
France,10,-9
